## Sequences vs Classes in PyTorch
Alternatively to nn.Sequential there is another commonly method of model definition an that is using a model class.

### Defining neural networks as sequences
In Pytorch the representation of a neural networks can be made as a sequence of layers. Each layer performs a specific computation on the input data and passes the transformed output to the next layer. 

As example:

```
import torch
import torch.nn as nn

# Define the neural network architecture
model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
    nn.LogSoftmax(dim=1)
)
```

This neural network have 3 dense layers interspersed with ReLU activation functions. The final layer utilizes the LogSoftMax activation commonly used in classification tasks.

### Defining Neural Networks With classes
nn.Sequential approach is intuitive and convinient in many cases but it can become limiting when we need more flexibility in the network achitecture. Defining neural networks as classes allows us to create custom architectures with complex behaviours and shareable components.

To define a neural network as a class, we typically subclass the nn.Module provided by PyTorch. This base class offers essential functionality for organizing the network's parameters and handling computations during forward and backward passes.

```
import torch
import torch.nn as nn

class CustomNetwork(nn.Module):
    def __init__(self):
        super(CustomNetwork, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.log_softmax(x)
        return x

# Create an instance of the CustomNetwork
model = CustomNetwork()        
```
In this example, we define a class called CustomNetwork that inherits from nn.Module. Inside the class, we describe the network's layers as attributes. The forward method specifies how the input flows through these layers during the forward pass.

By defining neural networks as classes, we can create more complex architectures, leverage conditional logic within the network, and encapsulate reusable components. This flexibility becomes particularly useful as we delve into advanced topics throughout the course.



### Class Example


In [11]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

In [12]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Net, self).__init__()
        
        # Define each of the layers
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 25)
        self.layer3 = nn.Linear(25, output_dim)
        
    def forward(self, x):
        # pass the input through the layers
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    

In [28]:
def fill_na_with_median(df):
    """
    Replace all na values in numeric fields with median 
    """
    for col in df.columns:
        if df[col].dtype in ['float32', 'int32', 'int64', 'float64', 'long']:
            median = df[col].median()
            df[col].fillna(median, inplace=True)
    return df

def load_auto_mpg():
    df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])
    df = fill_na_with_median(df)
    return df

df = load_auto_mpg()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   year          398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.1+ KB


In [35]:
# creating tensors for regression model
def regression_tensors(df, feature_columns, target_column, default_device="cpu"):
    x = torch.tensor(df[feature_columns].values, device=default_device, dtype=torch.float32)
    y = torch.tensor(df[target_column].values, device = default_device, dtype=torch.float32)
    return x, y

x, y = regression_tensors(df, feature_columns=[
            "cylinders",
            "displacement",
            "horsepower",
            "weight",
            "acceleration",
            "year",
            "origin",
        ], target_column="mpg")

In [36]:
# setting the model for regression task 
# model instantiation
device = "cpu"
model = Net(x.shape[1], 1).to(device)

# define the loss function
loss_fn = nn.MSELoss()

# define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)

In [38]:
# training loop 
for epoch in range(1000):
    # zero gradients
    optimizer.zero_grad()
    
    # forward pass
    y_pred = model.forward(x).flatten()
    
    # compute loss
    loss = loss_fn(y_pred, y)
    
    # backward pass
    loss.backward()
    
    # update parameters
    optimizer.step()
    
    if epoch % 5 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item():.4f}")

Epoch: 0, Loss: 624.1686
Epoch: 5, Loss: 623.5555
Epoch: 10, Loss: 622.9426
Epoch: 15, Loss: 622.3298
Epoch: 20, Loss: 621.7174
Epoch: 25, Loss: 621.1052
Epoch: 30, Loss: 620.4931
Epoch: 35, Loss: 619.8813
Epoch: 40, Loss: 619.2697
Epoch: 45, Loss: 618.6582
Epoch: 50, Loss: 618.0469
Epoch: 55, Loss: 617.4358
Epoch: 60, Loss: 616.8248
Epoch: 65, Loss: 616.2139
Epoch: 70, Loss: 615.6031
Epoch: 75, Loss: 614.9925
Epoch: 80, Loss: 614.3820
Epoch: 85, Loss: 613.7715
Epoch: 90, Loss: 613.1611
Epoch: 95, Loss: 612.5509
Epoch: 100, Loss: 611.9407
Epoch: 105, Loss: 611.3306
Epoch: 110, Loss: 610.7205
Epoch: 115, Loss: 610.1104
Epoch: 120, Loss: 609.5004
Epoch: 125, Loss: 608.8904
Epoch: 130, Loss: 608.2804
Epoch: 135, Loss: 607.6705
Epoch: 140, Loss: 607.0604
Epoch: 145, Loss: 606.4504
Epoch: 150, Loss: 605.8405
Epoch: 155, Loss: 605.2303
Epoch: 160, Loss: 604.6202
Epoch: 165, Loss: 604.0101
Epoch: 170, Loss: 603.3999
Epoch: 175, Loss: 602.7897
Epoch: 180, Loss: 602.1793
Epoch: 185, Loss: 601.5